<a href="https://colab.research.google.com/github/yosedie/ai-project/blob/main/ai_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ai-project

In [14]:
import numpy as np

# Representasi papan sebagai matriks
board = np.zeros((5, 5), dtype=int)

# Koordinat valid (bentuk papan permainan)
valid_positions = [
    (0, 0), (0, 2), (0, 4),
    (1, 1), (1, 3),
    (2, 0), (2, 1), (2, 2), (2, 3), (2, 4),
    (3, 1), (3, 3),
    (4, 0), (4, 2), (4, 4)
]

# Menampilkan papan
def print_board():
    for i in range(5):
        row = ""
        for j in range(5):
            if (i, j) not in valid_positions:
                row += "   "  # Posisi tidak valid
            elif board[i, j] == 0:
                row += " . "  # Kosong
            elif board[i, j] == 1:
                row += " M "  # Macan
            elif board[i, j] == 2:
                row += " U "  # Uwong
        print(row)
    print("\n")

# Cek apakah langkah valid
def is_valid_move(x, y):
    if (x, y) not in valid_positions:
        print(f"Invalid position: {(x, y)} is not a valid position.")
        return False
    if board[x, y] != 0:
        print(f"Invalid position: {(x, y)} is already occupied.")
        return False
    return True

# Peletakan awal pion
def place_piece(player, x, y):
    if (x, y) in valid_positions and board[x, y] == 0:
        board[x, y] = player
        return True
    return False

# Cek apakah melompati pion uwong
def can_jump(x1, y1, x2, y2):
    dx, dy = (x2 - x1) // 2, (y2 - y1) // 2  # Arah lompat
    mid_x, mid_y = x1 + dx, y1 + dy
    if board[mid_x, mid_y] != 2:
        print(f"Cannot jump: No uwong at middle position {(mid_x, mid_y)}.")
        return False
    return True

# Gerakan pion
def move_piece(player, x1, y1, x2, y2):
    print(f"Validating move for player {player} from {(x1, y1)} to {(x2, y2)}")
    if board[x1, y1] != player:  # Pastikan posisi awal benar
        print(f"Invalid move: Starting position does not belong to player {player}.")
        return False
    if not is_valid_move(x2, y2):  # Pastikan posisi tujuan valid
        print(f"Invalid move: Target position {(x2, y2)} is not valid.")
        return False
    if player == 1 and abs(x2 - x1) == 2 and abs(y2 - y1) == 2:  # Melompat
        if not can_jump(x1, y1, x2, y2):
            print(f"Invalid jump: No uwong to jump over from {(x1, y1)} to {(x2, y2)}.")
            return False
    elif abs(x2 - x1) > 1 or abs(y2 - y1) > 1:  # Gerakan biasa terlalu jauh
        print(f"Invalid move: Movement exceeds one step from {(x1, y1)} to {(x2, y2)}.")
        return False
    # Gerakan valid
    board[x1, y1] = 0
    board[x2, y2] = player
    return True

# Kondisi akhir permainan
def check_end_game():
    macan_positions = [(i, j) for i, j in valid_positions if board[i, j] == 1]
    uwong_positions = [(i, j) for i, j in valid_positions if board[i, j] == 2]

    if not uwong_positions:  # Tidak ada uwong tersisa
        print("Macan menang! Semua uwong telah diterkam.")
        return True

    # Cek apakah macan tidak bisa bergerak
    for mx, my in macan_positions:
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1), (-2, 0), (2, 0), (0, -2), (0, 2)]:
            nx, ny = mx + dx, my + dy
            if is_valid_move(nx, ny):
                return False
            if dx % 2 == 0 and can_jump(mx, my, nx, ny):
                return False

    print("Uwong menang! Semua macan tidak dapat bergerak.")
    return True

# Logika giliran
def player_turn(player):
    print_board()
    print("Giliran Macan" if player == 1 else "Giliran Uwong")
    try:
        x, y = map(int, input("Masukkan posisi awal (x y): ").split())
        if player == 1:  # Macan bergerak
            x2, y2 = map(int, input("Masukkan posisi tujuan (x y): ").split())
            if move_piece(player, x, y, x2, y2):
                return True
        else:  # Uwong meletakkan pion
            if place_piece(player, x, y):
                return True
    except ValueError:
        print("Input invalid. Masukkan angka dengan format 'x y'.")
    print("Langkah tidak valid, coba lagi.")
    return False


def is_valid_move_macan(start, target, board):
    # Pastikan target kosong
    if board[target[0]][target[1]] != ".":
        return False
    # Pastikan langkah valid (1 langkah atau melompat pion uwong)
    dx, dy = abs(start[0] - target[0]), abs(start[1] - target[1])
    if dx <= 1 and dy <= 1:  # 1 langkah
        return True
    if dx > 1 or dy > 1:  # Melompat
        if (dx == dy or dx == 0 or dy == 0):  # Dalam satu garis
            # Cek jumlah pion uwong di jalur
            uwong_count = count_uwong_in_path(start, target, board)
            return uwong_count % 2 == 0  # Harus genap
    return False

# Permainan utama
def main():
    uwong_count = 8
    placed_uwong = 0
    macan_count = 2
    current_player = 1  # Macan memulai permainan

    # Peletakan awal pion macan
    print("Letakkan pion Macan pertama:")
    while not place_piece(1, 0, 0):  # Misalnya, letakkan pion di (0, 0)
        pass
    print("Letakkan pion Macan kedua:")
    while not place_piece(1, 2, 2):  # Misalnya, letakkan pion kedua di (2, 2)
        pass

    while True:
        if current_player == 1:  # Giliran macan
            if not player_turn(1):
                continue
        else:  # Giliran uwong
            if placed_uwong < uwong_count:
                if not player_turn(2):
                    continue
                placed_uwong += 1
            else:  # Uwong habis, mulai bergerak
                if not player_turn(2):
                    continue

        # Cek kondisi akhir permainan
        if check_end_game():
            print_board()
            break

        # Ganti giliran
        current_player = 3 - current_player  # Bergantian 1 -> 2, 2 -> 1

if __name__ == "__main__":
    main()


Letakkan pion Macan pertama:
Letakkan pion Macan kedua:
 M     .     . 
    .     .    
 .  .  M  .  . 
    .     .    
 .     .     . 


Giliran Macan
Masukkan posisi awal (x y): 2 2
Masukkan posisi tujuan (x y): 3 3
Validating move for player 1 from (2, 2) to (3, 3)
Macan menang! Semua uwong telah diterkam.
 M     .     . 
    .     .    
 .  .  .  .  . 
    .     M    
 .     .     . 


